In [2]:
import qmcpy as qp
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import cProfile
import pstats

In [3]:
%matplotlib inline

In [4]:
def lattice(n, num_ports, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="natural" ,is_parallel=True)
    points = l.gen_samples(num_ports) 
    return points
def lattice_process(n, num_ports, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="natural_process")
    points = l.gen_samples(num_ports) 
    return points

In [12]:
trials = 2
d_list = [50, 100, 200, 300]
n = 10
lattice_d = {}
lattice_p_d = {}

run_list = [(lattice, "lattice"),(lattice_process,"lattice process")]
for method,method_name in run_list:
    
    for d in d_list:
        profiler = cProfile.Profile()
        profiler.enable()
        for _ in range(trials):
            _ = method(d, 2 ** n)
        profiler.disable()
        
        stats = pstats.Stats(profiler)
            
        process_time = stats.total_tt


        if method_name == "lattice":
            lattice_d.update({d:process_time})
        elif method_name == "lattice process":
            lattice_p_d.update({d:process_time})

print(lattice_d)
print(lattice_p_d)


{50: 0.015239600000000006, 100: 0.021601499999999996, 200: 0.03274850000000003, 300: 0.03845650000000001}
{50: 2.9136015000000004, 100: 2.7939316999999995, 200: 2.9488424000000006, 300: 2.9508294000000004}
